In [ ]:
from wayne_utils import load_data, save_data
import os


_ROOT_PATH = "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T"


test_text_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/data/CPL/dynamic/texts_test.json", "json")
train_text_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/data/CPL/dynamic/texts_train.json", "json")
test_label_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/data/CPL/dynamic/label_test.pickle", "pickle")
train_label_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/data/CPL/dynamic/label_train.pickle", "pickle")

train_count_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/evaluation/CountCPL/label_count_list_train.json", "json")

## 提示词指令

In [ ]:
general_instruction_22 = """根据给定的法律文本, 请你列出下面这些实例的数量: 
1.法院名称，
2.原告名称，
3.被告名称，
4.法院裁定的借款凭证，
5.法院裁定的约定的借款金额，
6.法院裁定的约定的还款日期或借款期限，
7.法院裁定的约定的利息，
8.法院裁定的约定的逾期利息，
9.法院裁定的约定的违约金，
10.原告诉称的借款凭证，
11.原告诉称的约定的借款金额，
12.原告诉称的约定的还款日期或借款期限，
13.原告诉称的约定的利息，
14.原告诉称的约定的逾期利息，
15.原告诉称的约定的违约金，
"""
general_instruction = """你是一名律师助手，请你根据给定的法律文本提取信息。 """
stage_1_instruction = """请从以下段落中提取所有相关事件，并以（ 角色类型, 字段, 取值）的三元组形式输出所有实例，要求如下：
（1）角色类型限定为："法院" 或 "原告" 或 "被告"。
（2）字段限定为：姓名名称、借款凭证的名称、借款凭证的所载内容、约定的借款金额的约定情况、约定的借款金额的实际发生时间、约定的借款金额的金额（元）、约定的还款日期或借款期限的约定情况、约定的还款日期或借款期限的实际发生时间、约定的还款日期或借款期限的还款日期、约定的利息的约定情况、约定的利息的实际发生时间、约定的利息的利率数值（百分比或元）、约定的逾期利息的约定情况、约定的逾期利息的实际发生时间、约定的逾期利息的利率数值（百分比或元）、约定的违约金的约定情况、约定的违约金的实际发生时间、约定的违约金的违约金数值（百分比或元）；
（3）除了姓名名称外，上述字段需要前缀注明是第几份/次，例如，借款凭证的名称应该写为第x份借款凭证的名称，以此类推。
（4）取值限定：所有日期相关的字段的取值格式应该为xxxx年xx月xx日，所有金额相关的取值应该统一以元为单位（如5万元应该写为50000元），约定情况只能从口头约定和书面约定中二选一，利率则取值限定为百分号前的数值。
（5）如果文中没有事件的相关信息，则不要呈现三元组。
现在，请提取出下面文字中所有的实例三元组：
"""

In [ ]:
def prompt_construct( texts_list, labels_list, mode):
    prompt_list = []
    for i in range( len(texts_list)):
        texts = texts_list[str(i)]
        labels = labels_list[i]
        if mode == "test":
            prompt_list.append( general_instruction + stage_1_instruction + str(texts) )
        else:
            prompt_list.append( {
                "instruction" : general_instruction,
                "input" : stage_1_instruction + str(texts),
                "output": str(labels)
            })
    return prompt_list

mode = "test"   # "train"
texts_list = test_text_list if mode == "test" else train_text_list
labels_list = test_label_list if mode == "test" else train_label_list
prompt_list = prompt_construct( texts_list, labels_list, mode)
save_data( prompt_list, os.path.join( "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T", "CPL_prompt_list.pickle"))

mode = "train"   # "train"
texts_list = test_text_list if mode == "test" else train_text_list
labels_list = test_label_list if mode == "test" else train_label_list
prompt_list = prompt_construct( texts_list, labels_list, mode)
save_data( prompt_list, os.path.join( "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T", "CPL_TTT_ft_list.json"))

## Post and Eval

In [ ]:
import re
import os
import sys
_ROOT_PATH = "/home/jiangpeiwen2/jiangpeiwen2/TKGT"
sys.path.insert( 0, _ROOT_PATH )
from evaluation.Text2table.evaluate import eval_main

input_string = "这是一个包含(苹果, 香蕉, 橘子)和(狗, 猫, 鱼)等三元组的字符串"
def triplet_get( input_string ):
    temp_set = set()
    pattern = r'\(([^,]+), ([^,]+), ([^,]+)\)'
    matches = re.findall(pattern, input_string)
    for match in matches:
        triplet = tuple(match)
        new_triplet = ()
        for item in triplet:
            if item.startswith("'"):
                item = item[1:]
            if item.endswith("'"):
                item = item[:-1]
            new_triplet = new_triplet + (item, )
        temp_set.add(new_triplet)
    return temp_set

model_list_table = [
    "ChatGLM3-6B",
    "Qwen1.5-7B-Chat",
    "Baichuan2-7B-Chat",
    "Chinese-Mistral-7B-Instruct-v0.1",
    "Qwen2.5-0.5B",
    "CPL_dynamic_tabel_TTT_ChatGLM3-6B-5epoch",
    "CPL_dynamic_tabel_Chinese-Mistral-7B-Instruct-v0.1-5epoch"
]
scopes = model_list_table[5:6]
scopes

In [ ]:
predict_list_raw = load_data( os.path.join( "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T", f"CPL_predict_listCPL_dynamic_tabel_TTT_ChatGLM3-6B-5epoch.pickle"), "pickle")

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
for model_name in scopes:
    predict_list_raw = load_data( os.path.join( "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T", f"CPL_predict_list{model_name}.pickle"), "pickle")
    labels_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/data/CPL/dynamic/label_test.pickle", "pickle")
    predict_lists = []
    for i in range( len(predict_list_raw)):
        predict_lists.append( triplet_get( predict_list_raw[i][0] ) )
    results_save_path = "/home/jiangpeiwen2/jiangpeiwen2/TKGT/evaluation/CountCPL/results.json"
    save_data( predict_lists, os.path.join( "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T", f"CPL_predict_list_filter_{model_name}.pickle"))
    save_data( (labels_list, predict_lists), os.path.join( "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T", f"CPL_pair_filter_{model_name}.pickle"))
    print( model_name )
    eval_main( f"CPL_dynamic_TTT_{model_name}", (labels_list, predict_lists), results_save_path, "multi_entity" )